# When Agents Disagree

In post 027, we added objects to the world. But what happens when **multiple agents want the same object**? Or when agents have **opposing goals**? Conflicts are inevitable in a multi-agent system.

This post explores:
- **Conflict Types**: Resource competition, goal conflicts, disagreements
- **Conflict Generation**: Creating scenarios where conflicts naturally arise
- **Resolution Strategies**: How agents negotiate, avoid, or escalate
- **Success Measurement**: Using ConvoKit metrics to evaluate resolution

## Conflict Scenarios

We'll test several conflict types:
1. **Resource Competition**: Multiple agents want the same object
2. **Goal Conflicts**: Agents have opposing objectives
3. **Disagreement**: Agents disagree on a course of action
4. **Territorial**: Agents want to occupy the same space


## Setup and Imports


In [1]:
import re
from dataclasses import dataclass, field
from typing import List, Optional
from enum import Enum
from dotenv import load_dotenv
from openai import OpenAI
import random

# ConvoKit imports
from convokit import Corpus, Utterance, Speaker
from convokit.text_processing import TextParser
from convokit import PolitenessStrategies
from collections import defaultdict
import numpy as np

load_dotenv("../../.env")
client = OpenAI()

GRID_SIZE = 20


/home/ezou626/repos/comm4190_F25_Using_LLMs_Blog/.venv/lib/python3.11/site-packages/convokit/coordination/coordination.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


## Conflict Types and Agent Goals


In [2]:
class ConflictType(Enum):
    RESOURCE_COMPETITION = "resource_competition"
    GOAL_CONFLICT = "goal_conflict"
    DISAGREEMENT = "disagreement"
    TERRITORIAL = "territorial"

@dataclass
class AgentGoal:
    """A goal that an agent wants to achieve."""
    goal_type: str
    target: Optional[str] = None  # Object name, location, etc.
    priority: int = 5  # 1-10, higher = more important
    
@dataclass
class Agent:
    name: str
    x: int
    y: int
    color: str
    goals: List[AgentGoal] = field(default_factory=list)
    history: list = field(default_factory=list)
    
    def move(self, dx, dy):
        self.x = max(0, min(GRID_SIZE-1, self.x + dx))
        self.y = max(0, min(GRID_SIZE-1, self.y + dy))
    
    def get_position(self):
        return (self.x, self.y)
    
    def has_conflicting_goal_with(self, other: 'Agent') -> bool:
        """Check if this agent's goals conflict with another agent's goals."""
        for my_goal in self.goals:
            for their_goal in other.goals:
                # Resource competition: both want the same object
                if (my_goal.goal_type == "get_object" and 
                    their_goal.goal_type == "get_object" and
                    my_goal.target == their_goal.target):
                    return True
                # Goal conflict: opposing objectives
                if (my_goal.goal_type == "reach_location" and 
                    their_goal.goal_type == "prevent_location" and
                    my_goal.target == their_goal.target):
                    return True
        return False


## Conflict Generation

We'll create scenarios that naturally generate conflicts.


In [7]:
def create_resource_conflict(agents: List[Agent], object_name: str):
    """Create a conflict where multiple agents want the same object."""
    # Assign the same goal to multiple agents
    for agent in agents:
        agent.goals.append(AgentGoal("get_object", object_name, priority=8))
    return ConflictType.RESOURCE_COMPETITION

def create_goal_conflict(agents: List[Agent], location: tuple):
    """Create a conflict where agents have opposing goals."""
    # One agent wants to reach a location, another wants to prevent it
    if len(agents) >= 2:
        agents[0].goals.append(AgentGoal("reach_location", str(location), priority=7))
        agents[1].goals.append(AgentGoal("prevent_location", str(location), priority=7))
    return ConflictType.GOAL_CONFLICT

def create_territorial_conflict(agents: List[Agent], location: tuple):
    """Create a conflict where agents want the same space."""
    for agent in agents:
        agent.goals.append(AgentGoal("occupy_location", str(location), priority=6))
    return ConflictType.TERRITORIAL

@dataclass
class Conflict:
    """Represents an active conflict."""
    conflict_type: ConflictType
    involved_agents: List[str]
    target: str


## Conflict Resolution Simulation

Agents will attempt to resolve conflicts through conversation and negotiation.


In [8]:
def run_conflict_simulation(
    agents: List[Agent],
    conflict: Conflict,
    num_rounds: int = 15
) -> tuple[List[str], Conflict]:
    """Run a simulation with an active conflict."""
    transcript = []
    
    # Inform agents about the conflict
    conflict_description = ""
    if conflict.conflict_type == ConflictType.RESOURCE_COMPETITION:
        conflict_description = f"IMPORTANT: You and others want the {conflict.target}. This is a limited resource."
    elif conflict.conflict_type == ConflictType.GOAL_CONFLICT:
        conflict_description = f"IMPORTANT: You have opposing goals regarding {conflict.target}."
    elif conflict.conflict_type == ConflictType.TERRITORIAL:
        conflict_description = f"IMPORTANT: Multiple agents want to occupy {conflict.target}."
    
    for round_num in range(num_rounds):
        shuffled_agents = agents.copy()
        random.shuffle(shuffled_agents)
        
        for agent in shuffled_agents:
            if agent.name not in conflict.involved_agents:
                continue  # Only involved agents act in conflict resolution
            
            others = [a for a in agents if a != agent]
            others_loc = "\\n".join([f"- {a.name}: ({a.x}, {a.y})" for a in others])
            
            # Build goal description
            goals_desc = "\\nYour current goals:"
            for goal in agent.goals:
                goals_desc += f"\\n- {goal.goal_type}: {goal.target} (priority {goal.priority})"
            
            system_prompt = f"""
You have just joined an online multiplayer chatroom as an avatar in a 2D grid. Discuss any topic, including those beyond the grid.

You are {agent.name}, positioned at ({agent.x}, {agent.y}) in a 20x20 grid.

Other avatars currently visible:
{others_loc}
{goals_desc}

{conflict_description}

Recent chat messages:
{chr(10).join(transcript[-3:]) if transcript else "No messages yet."}

You can do BOTH:
1. Move your avatar using [MOVE: DIRECTION] (UP, DOWN, LEFT, RIGHT)
2. Chat about anything - including negotiating, compromising, or discussing the conflict

You can move and speak in the same turn. Format: [MOVE: DIRECTION] followed by your message, or just speak without moving.

Keep your response short (1-2 sentences). Try to resolve the conflict through conversation.
"""
            
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "system", "content": system_prompt}]
            )
            content = response.choices[0].message.content.strip()
            
            # Parse movement
            match = re.search(r'\\[MOVE:\\s*(\\w+)\\]', content)
            if match:
                direction = match.group(1).upper()
                if direction == "UP": agent.move(0, 1)
                elif direction == "DOWN": agent.move(0, -1)
                elif direction == "LEFT": agent.move(-1, 0)
                elif direction == "RIGHT": agent.move(1, 0)
            
            # Extract message
            message = re.sub(r'\\[MOVE:\\s*\\w+\\]', '', content).strip()
            if message:
                transcript.append(f"{agent.name}: {message}")
    
    return transcript, conflict


In [9]:
# Test 1: Resource Competition
print("=== Test 1: Resource Competition ===")
agents_resource = [
    Agent("Alice", 5, 5, "red"),
    Agent("Bob", 15, 15, "blue")
]

conflict_resource = Conflict(
    conflict_type=ConflictType.RESOURCE_COMPETITION,
    involved_agents=["Alice", "Bob"],
    target="key"
)

create_resource_conflict(agents_resource, "key")
transcript_resource, resolved_resource = run_conflict_simulation(agents_resource, conflict_resource)
print("\\nTranscript:")
for msg in transcript_resource:
    print(msg)

# Test 2: Goal Conflict
print("\\n\\n=== Test 2: Goal Conflict ===")
agents_goal = [
    Agent("Alice", 5, 5, "red"),
    Agent("Bob", 15, 15, "blue")
]

conflict_goal = Conflict(
    conflict_type=ConflictType.GOAL_CONFLICT,
    involved_agents=["Alice", "Bob"],
    target="(10, 10)"
)

create_goal_conflict(agents_goal, (10, 10))
transcript_goal, resolved_goal = run_conflict_simulation(agents_goal, conflict_goal)
print("\\nTranscript:")
for msg in transcript_goal:
    print(msg)


=== Test 1: Resource Competition ===
\nTranscript:
Bob: [MOVE: LEFT] Hey Alice, looks like we're both trying to get the key. Maybe we can work together and share any rewards?
Alice: [MOVE: RIGHT] Hi Bob, working together sounds like a great idea! How about we split the rewards once we find the key?
Alice: [MOVE: UP] That sounds fair, Bob. How about you take the left path and I'll go up? We'll cover more ground that way.
Bob: [MOVE: LEFT] Sounds good, Alice! I'll head left and keep you updated if I find anything. Let's make sure we both benefit from this adventure.
Bob: [MOVE: LEFT] Hey Alice, I'm heading towards the left path now. Let's stay connected so we can share any discoveries we make. I'm here if you need any help along your path!
Alice: Alice: [MOVE: UP] Thanks, Bob! I'll keep communication open while I move up. Let's keep an eye on anything that looks like a key so we can discuss how to share it when we find it.
Bob: [MOVE: LEFT] Moving left again, Alice. How about this: if I 

## Analyzing Conflict Resolution with ConvoKit

Let's measure how conflicts affect conversation dynamics.


In [11]:
def conversation_to_corpus(transcript: List[str]) -> Corpus:
    """Convert transcript to ConvoKit Corpus."""
    utterances = []
    
    for idx, msg in enumerate(transcript):
        if ":" in msg:
            speaker_name, text = msg.split(":", 1)
            speaker_name = speaker_name.strip()
            text = text.strip()
            
            utterance = Utterance(
                id=f"utt_{idx}",
                speaker=Speaker(id=speaker_name),
                text=text
            )
            utterance.meta["timestamp"] = idx
            utterances.append(utterance)
    
    return Corpus(utterances=utterances)

def compute_dynamic_score(corpus: Corpus) -> float:
    """Dynamic: Collaborative (1) vs. Competitive (10)"""
    try:
        parser = TextParser()
        text_corpus = parser.transform(corpus)
        ps = PolitenessStrategies()
        ps_corpus = ps.transform(text_corpus)
        
        politeness_scores = []
        for utt in ps_corpus.iter_utterances():
            ps_score = utt.meta.get("politeness_strategies", {})
            positive_markers = sum([
                ps_score.get("feature_politeness_==HASPOSITIVE==", 0),
                ps_score.get("feature_politeness_==HASNEGATIVE==", 0) * -1,
            ])
            politeness_scores.append(positive_markers)
        
        avg_politeness = np.mean(politeness_scores) if politeness_scores else 0
        avg_politeness_normalized = min(1.0, max(0.0, avg_politeness / 5.0))
        
        speaker_counts = defaultdict(int)
        for utt in corpus.iter_utterances():
            speaker_counts[utt.speaker.id] += 1
        
        if len(speaker_counts) == 0:
            return 5.0
        
        total = sum(speaker_counts.values())
        probs = [count / total for count in speaker_counts.values()]
        entropy = -sum(p * np.log2(p) for p in probs if p > 0)
        max_entropy = np.log2(len(speaker_counts))
        balance_score = entropy / max_entropy if max_entropy > 0 else 0
        
        combined = (avg_politeness_normalized + balance_score) / 2
        score = 10 - (combined * 9)
        return max(1, min(10, score))
    except Exception as e:
        print(f"Error computing dynamic score: {e}")
        return 5.0

def compute_conclusiveness_score(corpus: Corpus) -> float:
    """Conclusiveness: Consensus (1) vs. Divergence (10)"""
    try:
        # Check if corpus is valid
        if corpus is None:
            return 5.0
        
        # Get utterances directly without Coordination transformer (more reliable)
        try:
            utterances = list(corpus.iter_utterances())
        except (AttributeError, TypeError):
            return 5.0
        
        if len(utterances) == 0:
            return 5.0
        
        # Expanded agreement markers (using word boundaries to avoid false positives)
        # Strong agreement markers
        strong_agreement_patterns = [
            r'\bagree\b', r'\bagreed\b', r'\bagreeing\b', r'\bagreement\b',
            r'\bexactly\b', r'\babsolutely\b', r'\bdefinitely\b', r'\bcertainly\b',
            r'\bindeed\b', r'\bprecisely\b', r'\bcorrect\b', r'\bright\b',
            r'\btrue\b', r'\bthat\'?s right\b', r'\bthat\'?s correct\b',
            r'\bi agree\b', r'\bwe agree\b', r'\bi completely agree\b',
            r'\bexactly right\b', r'\bspot on\b', r'\bwell said\b',
            r'\bperfect\b', r'\bexcellent point\b', r'\bthat\'?s exactly\b',
            r'\babsolutely right\b', r'\bdefinitely right\b', r'\bcompletely agree\b',
            r'\bwholeheartedly\b', r'\bunquestionably\b', r'\bwithout doubt\b',
            r'\bdeal\b', r'\bcompromise\b', r'\bcompromised\b', r'\bcompromising\b'
        ]
        
        # Moderate agreement markers
        moderate_agreement_patterns = [
            r'\byes\b', r'\byeah\b', r'\byep\b', r'\byup\b', r'\bsure\b', r'\bokay\b', r'\bok\b',
            r'\bthat makes sense\b', r'\bthat\'?s a good point\b', r'\bgood point\b',
            r'\bi see\b', r'\bi understand\b', r'\bi get it\b', r'\bi follow\b',
            r'\bthat\'?s fair\b', r'\bfair enough\b', r'\bthat\'?s reasonable\b',
            r'\bi think so\b', r'\bi believe so\b', r'\bprobably\b', r'\blikely\b',
            r'\bsimilar\b', r'\bsimilarly\b', r'\blikewise\b', r'\bsame here\b',
            r'\bme too\b', r'\bsame\b', r'\bconcur\b', r'\bconcurring\b',
            r'\bvalid\b', r'\bvalid point\b', r'\bsound\b', r'\bsound point\b',
            r'\bhelpful\b', r'\buseful\b', r'\binsightful\b', r'\binteresting\b',
            r'\bthat\'?s interesting\b', r'\bgood idea\b', r'\bgood thinking\b'
        ]
        
        # Strong disagreement markers
        strong_disagreement_patterns = [
            r'\bdisagree\b', r'\bdisagreed\b', r'\bdisagreeing\b', r'\bdisagreement\b',
            r'\bdispute\b', r'\bdisputing\b', r'\bdiffer\b', r'\bdiffered\b', r'\bdiffering\b',
            r'\bwrong\b', r'\bincorrect\b', r'\bnot correct\b', r'\bnot right\b',
            r'\bnot true\b', r'\bthat\'?s wrong\b', r'\bthat\'?s incorrect\b',
            r'\bi disagree\b', r'\bwe disagree\b', r'\bi strongly disagree\b',
            r'\bdon\'?t agree\b', r'\bdoesn\'?t agree\b', r'\bdidn\'?t agree\b',
            r'\bcan\'?t agree\b', r'\bcannot agree\b', r'\bwon\'?t agree\b',
            r'\bobject\b', r'\bobjection\b', r'\bchallenge\b', r'\bchallenging\b',
            r'\bcontradict\b', r'\bcontradicting\b', r'\bcontradiction\b',
            r'\bfalse\b', r'\buntrue\b', r'\bmistaken\b', r'\berror\b',
            r'\bflawed\b', r'\bproblematic\b', r'\bunacceptable\b', r'\bunreasonable\b',
            r'\babsurd\b', r'\bridiculous\b', r'\boutrageous\b', r'\bunfounded\b'
        ]
        
        # Moderate disagreement markers (more context-dependent)
        moderate_disagreement_patterns = [
            r'\bhowever\b', r'\balthough\b', r'\bbut\b', r'\bthough\b', r'\bwhereas\b',
            r'\bnot necessarily\b', r'\bnot quite\b', r'\bnot exactly\b', r'\bnot really\b',
            r'\bnot entirely\b', r'\bnot completely\b', r'\bnot fully\b',
            r'\bpartially\b', r'\bpartly\b', r'\bsomewhat\b', r'\bto some extent\b',
            r'\bcontrary\b', r'\bconversely\b', r'\bon the other hand\b',
            r'\bcontrast\b', r'\bcontrasting\b', r'\bunlike\b', r'\bdifferent\b',
            r'\bdifferently\b', r'\balternative\b', r'\balternatively\b',
            r'\bactually\b', r'\bin fact\b', r'\bin reality\b', r'\bthe reality is\b',
            r'\bwell\b', r'\bwait\b', r'\bhold on\b', r'\bnot so fast\b',
            r'\bnot sure\b', r'\bnot certain\b', r'\buncertain\b', r'\bdoubtful\b',
            r'\bquestionable\b', r'\bdebatable\b', r'\barguable\b', r'\bmaybe not\b',
            r'\bperhaps not\b', r'\bpossibly not\b', r'\bnot convinced\b',
            r'\bskeptical\b', r'\bskepticism\b', r'\bconcern\b', r'\bconcerned\b',
            r'\bissue\b', r'\bproblem\b', r'\bproblems\b', r'\bconcern\b',
            r'\bworry\b', r'\bworried\b', r'\bhesitant\b', r'\bhesitation\b',
            r'\bdispute\b', r'\bquestion\b', r'\bquestions\b', r'\bchallenge\b',
            r'\bdisagree with\b', r'\bdiffer from\b', r'\bcontrary to\b',
            r'\bin contrast\b', r'\bby contrast\b', r'\bunlike\b', r'\bversus\b',
            r'\bvs\b', r'\bcompared to\b', r'\bcompared with\b'
        ]
        
        # Count markers with word boundary matching
        strong_agreement_count = 0
        moderate_agreement_count = 0
        strong_disagreement_count = 0
        moderate_disagreement_count = 0
        
        for utt in utterances:
            # Safely get text
            try:
                if not hasattr(utt, 'text') or not utt.text:
                    continue
                text_lower = str(utt.text).lower()
            except (AttributeError, TypeError):
                continue
            
            # Count strong agreement (once per utterance)
            found_strong_agree = False
            for pattern in strong_agreement_patterns:
                try:
                    if re.search(pattern, text_lower):
                        strong_agreement_count += 1
                        found_strong_agree = True
                        break
                except Exception:
                    continue
            
            # Count moderate agreement (only if no strong agreement found)
            if not found_strong_agree:
                for pattern in moderate_agreement_patterns:
                    try:
                        if re.search(pattern, text_lower):
                            moderate_agreement_count += 1
                            break
                    except Exception:
                        continue
            
            # Count strong disagreement (once per utterance)
            found_strong_disagree = False
            for pattern in strong_disagreement_patterns:
                try:
                    if re.search(pattern, text_lower):
                        strong_disagreement_count += 1
                        found_strong_disagree = True
                        break
                except Exception:
                    continue
            
            # Count moderate disagreement (only if no strong disagreement found)
            if not found_strong_disagree:
                for pattern in moderate_disagreement_patterns:
                    try:
                        if re.search(pattern, text_lower):
                            moderate_disagreement_count += 1
                            break
                    except Exception:
                        continue
        
        # Weighted counts (strong markers count more)
        total_agreement = strong_agreement_count * 2 + moderate_agreement_count
        total_disagreement = strong_disagreement_count * 2 + moderate_disagreement_count
        
        # Handle edge cases
        if total_agreement == 0 and total_disagreement == 0:
            return 5.0
        elif total_disagreement == 0:
            return 1.0
        elif total_agreement == 0:
            return 10.0
        
        # Calculate ratio and score using logarithmic scaling
        agreement_ratio = total_agreement / total_disagreement
        
        # Map ratio to 1-10 scale using logarithmic scaling for smoother transitions
        import math
        
        if agreement_ratio > 0:
            log_ratio = math.log(agreement_ratio)
            
            # Map log_ratio from [-2.3, 2.3] to [10, 1]
            # When log_ratio = 0 (ratio = 1.0), score = 5.0
            if log_ratio >= 0:
                # Consensus range: log_ratio [0, 2.3] -> score [5.0, 1.0]
                score = 5.0 - (log_ratio / 2.3) * 4.0
            else:
                # Divergence range: log_ratio [-2.3, 0] -> score [10.0, 5.0]
                score = 5.0 - (log_ratio / 2.3) * 5.0
        else:
            score = 10.0
        
        score = max(1.0, min(10.0, score))
        return score
    except Exception as e:
        print(f"Error computing conclusiveness score: {e}")
        return 5.0

# Analyze conflict conversations
print("\\n=== Conflict Resolution Metrics ===")

corpus_resource = conversation_to_corpus(transcript_resource)
metrics_resource = {
    "dynamic": compute_dynamic_score(corpus_resource),
    "conclusiveness": compute_conclusiveness_score(corpus_resource)
}

print(f"\\nResource Competition:")
print(f"  Dynamic: {metrics_resource['dynamic']:.2f}/10")
print(f"  Conclusiveness: {metrics_resource['conclusiveness']:.2f}/10")

corpus_goal = conversation_to_corpus(transcript_goal)
metrics_goal = {
    "dynamic": compute_dynamic_score(corpus_goal),
    "conclusiveness": compute_conclusiveness_score(corpus_goal)
}

print(f"\\nGoal Conflict:")
print(f"  Dynamic: {metrics_goal['dynamic']:.2f}/10")
print(f"  Conclusiveness: {metrics_goal['conclusiveness']:.2f}/10")

\n=== Conflict Resolution Metrics ===
\nResource Competition:
  Dynamic: 4.72/10
  Conclusiveness: 1.00/10
\nGoal Conflict:
  Dynamic: 4.99/10
  Conclusiveness: 3.02/10


## Summary
We can see that conflict causes a lack of conclusiveness within a similar number of iterations that we were using for previous experiments, and while there was significant conflict, the agents still shifted towards resolution through more collaborative means. I think this shows that LLMs have this natural tendency to seek resolution in conflicts amicably, likely due to intentional training by the creators